In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PageRank").getOrCreate()
sc = spark.sparkContext
print(spark.version)


4.0.0


In [6]:
lines = sc.textFile("question2.txt")
links = lines.map(lambda x: x.split(":")).map(lambda x: (int(x[0]), [int(i) for i in eval(x[1])]))
links = links.cache()
links.take(5)

[(0, [25, 58, 6, 35, 47, 70, 26, 12, 73]),
 (1, [21, 60]),
 (2, [81, 57, 18, 82, 96]),
 (3, [62, 64, 14, 58, 48, 0, 56]),
 (4, [48, 54, 60, 97, 2])]

In [7]:
links = links.partitionBy(4).cache()
ranks = links.mapValues(lambda _: 1.0)

for iteration in range(10):
    contribs = links.join(ranks).flatMap(
        lambda node_neighbors_rank: [
            (neighbor, node_neighbors_rank[1][1] / len(node_neighbors_rank[1][0]))
            for neighbor in node_neighbors_rank[1][0]
        ]
    )
    ranks = contribs.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: 0.15 + 0.85 * rank)

output = ranks.collect()
for (node, rank) in output[:10]:
    print(f"Node {node} has rank: {rank:.4f}")



Node 12 has rank: 0.7758
Node 48 has rank: 0.9541
Node 60 has rank: 1.1857
Node 52 has rank: 0.8770
Node 80 has rank: 0.4671
Node 36 has rank: 1.3861
Node 40 has rank: 1.4596
Node 92 has rank: 0.6985
Node 64 has rank: 1.3443
Node 68 has rank: 0.4385


In [8]:
max_node = max(output, key=lambda x: x[1])
min_node = min(output, key=lambda x: x[1])

print(f"Highest rank: Node {max_node[0]} with rank {max_node[1]:.4f}")
print(f"Lowest rank: Node {min_node[0]} with rank {min_node[1]:.4f}")

Highest rank: Node 89 with rank 2.7494
Lowest rank: Node 24 with rank 0.2175
